In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from hook_manager import HookManager
from data_handling import load_tinystories_data
import random

import numpy as np
import os

%load_ext autoreload
%autoreload 2

In [2]:
model_name = "roneneldan/TinyStories-33M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [3]:
data = load_tinystories_data('data/tinystories_val.txt')

In [76]:
target = 'disappointed'
target_strings = []
non_target_strings = []

for i in range(len(data)):
    if target in data[i]:
        lo = max(0, data[i].find(target) - 30)
        hi = min(len(data[i]), data[i].find(target) + 100)
        target_strings.append(data[i][lo:hi])

    else:
        index = random.randint(0, len(data[i]))
        lo = max(0, index - 50)
        hi = min(len(data[i]), index + 100)
        non_target_strings.append(data[i][lo:hi])

In [77]:
for i, string_ in enumerate(target_strings):
    if i > 50:
        break
    print(string_)


it was empty. He felt sad and disappointed. Suddenly, he saw a bald eagle flying above him. The eagle swooped down and grabbed the
 parents. Lily's parents were disappointed and scolded her for taking something that wasn't hers. Lily learned that it's not right
 seeing the stars.  John felt disappointed and was not sure what to do. His dad knew they would have to wait until morning now and
ve a cell phone. Tam was very disappointed. Later that day, Tam's mom talked to her and said, "I have an idea that might be better
 was his. Katrine was sad and disappointed that she had to give away something of value. She knew nothing implressive would ever b
oo hard.  John and Milly were disappointed, so they demanded that mum find them some soft cauliflower.  Mum smiled and said she kn
tree rotting away. Max was so disappointed. He decided that he needed to do something.  He decided to take a nap to try and forget
it slipped away! Bob was very disappointed and searched for the thread everywhere. 

In [78]:
target_activations = []

with HookManager(model) as hook_manager:
    for layer in range(model.config.num_hidden_layers):
        target_activations.append(hook_manager.attach_residstream_hook(layer))
    
    for i, text in enumerate(target_strings[:200]):
        print(i)
        tokenized = tokenizer(text, return_tensors='pt')
        model(**tokenized)


non_target_activations = []

with HookManager(model) as hook_manager:
    for layer in range(model.config.num_hidden_layers):
        non_target_activations.append(hook_manager.attach_residstream_hook(layer))
    
    for i, text in enumerate(non_target_strings[:200]):
        print(i)
        tokenized = tokenizer(text, return_tensors='pt')
        model(**tokenized)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104


In [79]:
for layer in range(model.config.num_hidden_layers):
    layer_target = torch.concat(target_activations[layer], dim=0)
    layer_non_target = torch.concat(non_target_activations[layer], dim=0)

    mean_target = torch.mean(layer_target, dim=0)
    mean_non_target = torch.mean(layer_non_target, dim=0)

    mean_diff = mean_target - mean_non_target

    if not os.path.isdir(f'steering_vectors/tinystories_vectors/{target.strip()}'):
        os.mkdir(f'steering_vectors/tinystories_vectors/{target.strip()}')
    torch.save(mean_diff, f'steering_vectors/tinystories_vectors/{target.strip()}/layer_{layer}.pt')

In [44]:
mean_target = torch.mean(layer_2_target, dim=0)
mean_non_target = torch.mean(layer_2_non_target, dim=0)

mean_diff = mean_target - mean_non_target